In [23]:
import os
os.chdir(r'C:\Users\Hp\Desktop\ml labs\last labs\lab 2')

In [24]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
import sys
sys.path.append(r"C:\Users\Hp\Desktop\ml labs\last labs\timeseires")

from timeseires.utils.to_split import to_split
from timeseires.utils.multivariate_multi_step import multivariate_multi_step
from timeseires.utils.multivariate_single_step import multivariate_single_step
from timeseires.utils.univariate_multi_step import univariate_multi_step
from timeseires.utils.univariate_single_step import univariate_single_step
from timeseires.utils.CosineAnnealingLRS import CosineAnnealingLRS
from timeseires.callbacks.EpochCheckpoint import EpochCheckpoint
from tensorflow.keras.callbacks import ModelCheckpoint
from timeseires.callbacks.TrainingMonitor import TrainingMonitor
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Input,LSTM, Bidirectional, Add
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv1D,TimeDistributed
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,MaxPooling1D,Concatenate,AveragePooling1D, GlobalMaxPooling1D, Input
from tensorflow.keras.models import Sequential,Model
import pandas as pd
import time, pickle
import numpy as np
import tensorflow.keras.backend as K
import tensorflow
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanAbsoluteError
from tensorflow.keras.metrics import MeanAbsolutePercentageError
from tensorflow.keras.layers import Input, Reshape, Dense, Lambda
from tensorflow.keras.layers import Layer, Flatten, LeakyReLU, concatenate
from tensorflow.keras.regularizers import l2
import glob
import h5py
import matplotlib.pyplot as plt
from keras.callbacks import Callback

In [25]:
class PC:
    @staticmethod
    def build(time_steps, num_features, reg=0.0005):
        # initialize the input shape to be "channels last" and the
        # channels dimension itself
        # define the model input and first CONV module
        inputs = Input(shape=(time_steps, num_features))
        lstm=Bidirectional(LSTM(48,return_sequences=True))(inputs)
        lstm=LSTM(48)(lstm)
        lstm=Activation('tanh')(lstm)
        lstm=Dense(24)(lstm)
        # create the model
        model = Model(inputs, lstm, name="PC")
        
        # return the constructed network architecture
        return model

In [26]:
model1 =  PC.build(time_steps=24, num_features=21, reg=0.005)

In [27]:
model1.summary()

Model: "PC"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 24, 21)]          0         
                                                                 
 bidirectional_2 (Bidirectio  (None, 24, 96)           26880     
 nal)                                                            
                                                                 
 lstm_5 (LSTM)               (None, 48)                27840     
                                                                 
 activation_2 (Activation)   (None, 48)                0         
                                                                 
 dense_2 (Dense)             (None, 24)                1176      
                                                                 
Total params: 55,896
Trainable params: 55,896
Non-trainable params: 0
____________________________________________________________

In [28]:
import os
checkpoints = r'C:\Users\Hp\Desktop\ml labs\last labs\lab 2\checkpoints\\E3-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
OUTPUT_PATH = r'C:\Users\Hp\Desktop\ml labs\last labs\lab 2\checkpoints'
FIG_PATH = os.path.sep.join([OUTPUT_PATH,r"h2istory.png"])
JSON_PATH = os.path.sep.join([OUTPUT_PATH,r"h2istory.json"])

In [29]:
model = None
start_epoch = 0
time_steps=24
num_features=21

In [30]:
# construct the callback to save only the *best* model to disk
# based on the validation loss
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                             monitor="val_loss",
                             save_best_only=True, 
                             verbose=1)
TrainingMonitor1=TrainingMonitor(FIG_PATH, jsonPath=JSON_PATH, startAt=start_epoch)

# construct the set of callbacks
callbacks = [EpochCheckpoint1,TrainingMonitor1]

In [31]:
# if there is no specific model checkpoint supplied, initialize the network
if model is None:
    print("[INFO] compiling model...")
    model = PC.build(time_steps=24, num_features=21, reg=0.005)
    opt = Adam(1e-3)
    model.compile(loss='mae', optimizer=opt, metrics=["mae", "mape"])

# otherwise, load the checkpoint from disk
elif isinstance(model, str):  # Ensure model is a string path before loading
    if os.path.exists(model):  # Check if the file exists
        print(f"[INFO] loading model from: {model}")
        model = load_model(model)

        # update the learning rate
        print(f"[INFO] old learning rate: {K.get_value(model.optimizer.lr)}")
        K.set_value(model.optimizer.lr, 1e-4)
        print(f"[INFO] new learning rate: {K.get_value(model.optimizer.lr)}")
    else:
        raise FileNotFoundError(f"[ERROR] Model file not found: {model}")

else:
    print("[INFO] Model is already loaded, skipping loading step.")

[INFO] compiling model...


In [33]:
import os
path_dataset =r'C:\Users\Hp\Desktop\ml labs\last labs\AEP_hourly'
path_tr = os.path.join(path_dataset, 'AEP_train.csv')
df_tr = pd.read_csv(path_tr)
train_set = df_tr.iloc[:].values
path_v = os.path.join(path_dataset, 'AEP_validation.csv')
df_v = pd.read_csv(path_v)
validation_set = df_v.iloc[:].values 
path_te = os.path.join(path_dataset, 'AEP_test.csv')
df_te = pd.read_csv(path_te)
test_set = df_te.iloc[:].values 

path_scaler = os.path.join(path_dataset, 'AEP_Scaler.pkl')
scaler         = pickle.load(open(path_scaler, 'rb'))

train_set.shape, validation_set.shape, test_set.shape

((84907, 21), (24259, 21), (12130, 21))

In [34]:
time_steps=24
num_features=21

In [35]:
start = time.time()
train_X , train_y = univariate_multi_step(train_set, time_steps, target_col=0,target_len=24)
validation_X, validation_y = univariate_multi_step(validation_set, time_steps, target_col=0,target_len=24)
test_X, test_y = univariate_multi_step(test_set, time_steps, target_col=0,target_len=24)
print('Time Consumed', time.time()-start, "sec")

Time Consumed 1.1412959098815918 sec


In [36]:
epochs = 2
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,verbose = verbose)

Epoch 1/2
2652/2652 [==============================] - ETA: 0s - loss: 0.0463 - mae: 0.0463 - mape: 2305.0928
Epoch 1: val_loss improved from inf to 0.03953, saving model to C:\Users\Hp\Desktop\ml labs\last labs\lab 2\checkpoints\E3-cp-0001-loss0.04.h5
2652/2652 [==============================] - 120s 41ms/step - loss: 0.0463 - mae: 0.0463 - mape: 2305.0928 - val_loss: 0.0395 - val_mae: 0.0395 - val_mape: 17.5095
Epoch 2/2
2651/2652 [============================>.] - ETA: 0s - loss: 0.0364 - mae: 0.0364 - mape: 2202.1042
Epoch 2: val_loss did not improve from 0.03953
2652/2652 [==============================] - 85s 32ms/step - loss: 0.0364 - mae: 0.0364 - mape: 2201.4065 - val_loss: 0.0416 - val_mae: 0.0416 - val_mape: 18.4988


In [38]:
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanAbsoluteError

# Corrected file path
model_path = r"C:\Users\Hp\Desktop\ml labs\last labs\lab 2\checkpoints\E3-cp-0001-loss0.04.h5"

# Verify if the file exists
if not os.path.exists(model_path):
    raise FileNotFoundError(f"Error: File not found at {model_path}")

# Load model with custom metric handling
try:
    model = load_model(model_path, custom_objects={'mae': MeanAbsoluteError()})
    print("Model loaded successfully with custom metric!")
except TypeError:
    print("Warning: Could not locate custom function. Loading without compilation.")
    model = load_model(model_path, compile=False)

# Predictions
y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)

# Error metrics
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE):', round(MAE, 2))

MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE):', round(MEDAE, 2))

MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE):', round(MSE, 2))

RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE):', round(RMSE, 2))

MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred) / y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE):', round(MAPE, 2), '%')

MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred) / y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE):', round(MDAPE, 2), '%')

print('\n\ny_test_unscaled.shape =', y_test_unscaled.shape)
print('y_pred.shape =', y_pred.shape)


Model loaded successfully with custom metric!
378/378 [==============================] - 7s 13ms/step
Mean Absolute Error (MAE): 638.17
Median Absolute Error (MedAE): 506.61
Mean Squared Error (MSE): 686810.72
Root Mean Squared Error (RMSE): 828.74
Mean Absolute Percentage Error (MAPE): 4.32 %
Median Absolute Percentage Error (MDAPE): 3.53 %


y_test_unscaled.shape = (12082, 24)
y_pred.shape = (12082, 24)


In [41]:
checkpoints = r'C:\Users\Hp\Desktop\ml labs\last labs\lab 2\checkpoints\E2-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
model=r'C:\Users\Hp\Desktop\ml labs\last labs\lab 2\checkpoints\E3-cp-0001-loss0.04.h5'
start_epoch= 0

In [43]:
import os
import numpy as np
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanAbsoluteError
from tensorflow.keras.callbacks import ModelCheckpoint

# ✅ Correct import depending on TensorFlow version
try:
    from keras.saving import register_keras_serializable  # TF 2.12+
except ImportError:
    from keras.utils.generic_utils import register_keras_serializable  # TF < 2.12

# 🔹 Register custom metrics
@register_keras_serializable(package="Custom", name="mae")
def mae(y_true, y_pred):
    return MeanAbsoluteError()(y_true, y_pred)

@register_keras_serializable(package="Custom", name="mape")
def mape(y_true, y_pred):
    return K.mean(K.abs((y_true - y_pred) / K.clip(y_true, K.epsilon(), None))) * 100

# ✅ Model checkpoint path
checkpoint_path = r"C:\Users\Hp\Desktop\ml labs\last labs\lab 2\checkpoints\E3-cp-0001-loss0.04.h5"

# Check if file exists
if not os.path.exists(checkpoint_path):
    raise FileNotFoundError(f"❌ Error: Model checkpoint not found at {checkpoint_path}")

print(f"[INFO] Loading model from {checkpoint_path}...")

# 🔽 Load model with custom metrics
try:
    model = load_model(checkpoint_path, custom_objects={"mae": mae, "mape": mape}, compile=False)
    print("✅ Model loaded successfully!")
except Exception as e:
    print(f"❌ Error loading model: {e}")
    exit()

# 🔄 Recompile the model
print("[INFO] Recompiling the model with Adam optimizer...")
optimizer = Adam(learning_rate=1e-4)
model.compile(loss="mae", optimizer=optimizer, metrics=["mae", "mape"])

# 🔍 Show current learning rate
print("[INFO] New learning rate:", K.get_value(model.optimizer.learning_rate))

# 📊 Show model summary
print(model.summary())


[INFO] Loading model from C:\Users\Hp\Desktop\ml labs\last labs\lab 2\checkpoints\E3-cp-0001-loss0.04.h5...
✅ Model loaded successfully!
[INFO] Recompiling the model with Adam optimizer...
[INFO] New learning rate: 1e-04
Model: "PC"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 24, 21)]          0         
                                                                 
 bidirectional_3 (Bidirectio  (None, 24, 96)           26880     
 nal)                                                            
                                                                 
 lstm_7 (LSTM)               (None, 48)                27840     
                                                                 
 activation_3 (Activation)   (None, 48)                0         
                                                                 
 dense_3 (Dense)             (None, 24)  

In [44]:
epochs = 2
verbose = 1 #0
batch_size = 128
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,
                        verbose = verbose)

Epoch 1/2
663/663 [==============================] - ETA: 0s - loss: 0.0352 - mae: 0.0352 - mape: 2223.6260
Epoch 1: val_loss improved from 0.03953 to 0.03824, saving model to C:\Users\Hp\Desktop\ml labs\last labs\lab 2\checkpoints\E3-cp-0001-loss0.04.h5
663/663 [==============================] - 65s 80ms/step - loss: 0.0352 - mae: 0.0352 - mape: 2223.6260 - val_loss: 0.0382 - val_mae: 0.0382 - val_mape: 16.3774
Epoch 2/2
663/663 [==============================] - ETA: 0s - loss: 0.0347 - mae: 0.0347 - mape: 2230.7712
Epoch 2: val_loss improved from 0.03824 to 0.03740, saving model to C:\Users\Hp\Desktop\ml labs\last labs\lab 2\checkpoints\E3-cp-0002-loss0.04.h5
663/663 [==============================] - 55s 83ms/step - loss: 0.0347 - mae: 0.0347 - mape: 2230.7712 - val_loss: 0.0374 - val_mae: 0.0374 - val_mape: 15.8524


In [47]:
import os
import numpy as np
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanAbsoluteError

# ✅ Handle compatibility for register_keras_serializable
try:
    from keras.saving import register_keras_serializable  # TensorFlow ≥ 2.12
except ImportError:
    from keras.utils.generic_utils import register_keras_serializable  # TensorFlow < 2.12
    from keras.utils.generic_utils import _GLOBAL_CUSTOM_OBJECTS

    # Remove old if already registered
    _GLOBAL_CUSTOM_OBJECTS.pop("Custom>mae", None)
    _GLOBAL_CUSTOM_OBJECTS.pop("Custom>mape", None)

# 🔹 Register custom MAE
@register_keras_serializable(package="Custom", name="mae")
def mae(y_true, y_pred):
    return MeanAbsoluteError()(y_true, y_pred)

# 🔹 Register custom MAPE
@register_keras_serializable(package="Custom", name="mape")
def mape(y_true, y_pred):
    return K.mean(K.abs((y_true - y_pred) / K.clip(y_true, K.epsilon(), None))) * 100

# ✅ Model checkpoint path
checkpoint_path = r"C:\Users\Hp\Desktop\ml labs\last labs\lab 2\checkpoints\E3-cp-0001-loss0.04.h5"

if not os.path.exists(checkpoint_path):
    raise FileNotFoundError(f"❌ Error: Model checkpoint not found at {checkpoint_path}")

print(f"[INFO] Loading model from {checkpoint_path}...")

# 🔄 Load the model with custom metrics
try:
    model = load_model(checkpoint_path, custom_objects={"mae": mae, "mape": mape}, compile=False)
    print("✅ Model loaded successfully!")
except Exception as e:
    print(f"❌ Error loading model: {e}")
    exit()

# 🔁 Recompile the model
print("[INFO] Recompiling the model with Adam optimizer...")
optimizer = Adam(learning_rate=1e-4)
model.compile(loss="mae", optimizer=optimizer, metrics=["mae", "mape"])

# 📊 Model structure
print(model.summary())

# =================== PREDICTION & EVALUATION ===================

# ✅ Ensure test data and scaler are defined
if "test_X" not in locals() or "test_y" not in locals():
    raise ValueError("❌ Error: `test_X` or `test_y` is not defined.")
if "scaler" not in locals():
    raise ValueError("❌ Error: `scaler` is not defined for inverse_transform.")

# 🔍 Predict
print("[INFO] Making predictions...")
y_pred_scaled = model.predict(test_X)
y_pred = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)

# 📈 Evaluation Metrics
MAE = np.mean(abs(y_pred - y_test_unscaled))
MEDAE = np.median(abs(y_pred - y_test_unscaled))
MSE = np.mean(np.square(y_pred - y_test_unscaled))
RMSE = np.sqrt(MSE)
MAPE = np.mean(np.abs((y_test_unscaled - y_pred) / y_test_unscaled)) * 100
MDAPE = np.median(np.abs((y_test_unscaled - y_pred) / y_test_unscaled)) * 100

# 📤 Print Results
print('\n🔹 Evaluation Results:')
print('Mean Absolute Error (MAE):', np.round(MAE, 2))
print('Median Absolute Error (MedAE):', np.round(MEDAE, 2))
print('Mean Squared Error (MSE):', np.round(MSE, 2))
print('Root Mean Squared Error (RMSE):', np.round(RMSE, 2))
print('Mean Absolute Percentage Error (MAPE):', np.round(MAPE, 2), '%')
print('Median Absolute Percentage Error (MDAPE):', np.round(MDAPE, 2), '%')

print('\n🔸 Shapes:')
print('y_test_unscaled.shape:', y_test_unscaled.shape)
print('y_pred.shape:', y_pred.shape)


[INFO] Loading model from C:\Users\Hp\Desktop\ml labs\last labs\lab 2\checkpoints\E3-cp-0001-loss0.04.h5...
✅ Model loaded successfully!
[INFO] Recompiling the model with Adam optimizer...
Model: "PC"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 24, 21)]          0         
                                                                 
 bidirectional_3 (Bidirectio  (None, 24, 96)           26880     
 nal)                                                            
                                                                 
 lstm_7 (LSTM)               (None, 48)                27840     
                                                                 
 activation_3 (Activation)   (None, 48)                0         
                                                                 
 dense_3 (Dense)             (None, 24)                1176      
       